## Preliminaries

First, let's perform some basic installation and imports.

In [ ]:
%%capture 
%load_ext autoreload
%autoreload 2
! pip install ipywidgets

In [ ]:
from src.preprocess import NetflowPreprocessor
from src.guided_gae_model import GATEncoderWithEdgeAttr, GlobalEdgeEmbedding, DecoderWithGlobalEdge, GAEWithGlobalEdge
import cudf

In [ ]:
import torch 

torch.manual_seed(42)
torch.cuda.manual_seed(42)

Let's also look at what a sample of the netflow data we'll be using looks like. Please note that your data must follow the `NF-CICIDS 2018` data format to use the rest of the dataset as is. If your schema changes, please modify the code below to account for it, including parameters for the `NetflowPreprocessor`.

In [ ]:
## Load your train data and test data here into CuDF dataframes.
## You can use methods like `cudf.read_parquet` or `cudf.read_csv`

## Save some sample test data for the morpheus pipeline
## test_data.sample(n=1000).to_parquet('artifacts/sample_data.parquet')

## Step 1: Create Data Loaders for Graphs with Benign Training Data

This notebook is based on Netflow data from the Canadian Institue of Cybersecurity's CIC-IDS-2017 dataset. The data is collected across 5 working days (Monday-Friday) with Monday being all benign traffic, and the rest of the days containing mixed anomalous and benign traffic. 

In this example, we'll use Monday's benign Netflow to generate training examples. 

In [ ]:
# The first time this is run could take a few minutes 
processor = NetflowPreprocessor(
    train_data.head(n=1_000_000) if len(train_data) > 1_000_000 else train_data,
    edge_columns = ['IN_BYTES', 'OUT_BYTES', 'FLOW_DURATION_MILLISECONDS'],
    node_dim = 32,
)

Now that the preprocessor has been initialized, we can create training examples. We'll do this by splitting the netflow into `window_size`  long windows, with a step size of `step_size`. For each window, we'll create a directed graph containing the node properaties and edge properties, along with a label on every edge. Each of these will be a training example. The labels are irrelevant during training as this is a self-supervised problem.

In this case, all edge labels will be `0` because all connections are benign. All features will also be normalized using a CuML `StandardScaler`. 

Note that you will require a GPU to run these steps. Furthermore, you can also browse the implementation in the `src/preprocess.py` file.

In [ ]:
benign_graphs, benign_ip_map, benign_data_windows = processor.construct_graph_list(window_size=1000, step_size=1000)

Next, we'll split the graphs into training and validation splits, and construct PyTroch Geometric data loaders. 

In [ ]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split

def split_data(data_list, train_ratio=0.8):
    train_size = int(len(data_list) * train_ratio)
    test_size = len(data_list) - train_size
    train_dataset, test_dataset = random_split(data_list, [train_size, test_size])
    return train_dataset, test_dataset

# Assume benging_graphs is already created as shown previously
train_dataset, test_dataset = split_data(benign_graphs, train_ratio=0.8)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)


# You can also save the scaler for use in a Morpheus pipeline using the following line of code
# processor.save_scaler('artifacts/sample_edge_scaler.pkl') # Modify path as required

## Step 2: Construct Data Loaders for Malicious Test Data

Now, we'll create a data loader with data containing both benign and attack data. In this case, the edge attributes will also contain a label so that we can evaluate model performance down the line. 

Here, we won't re-instantiate the `preprocessor` object because it has already been built and contains scalers fit on benign data. This is important because we'll want to scale the test data using the scaler built on training data to avoid leakage.

In [ ]:
attack_graphs, attack_ip_map, attack_data_windows = processor.construct_graph_list(
    df = test_data,
    window_size = 1000,
    step_size=1000
)

attack_loader = DataLoader(attack_graphs, batch_size=256, shuffle=False)

## Step 3: Training the Graph Autoencoder (GAE)

In this section, we'll train a GAE to reconstruct the adjacency matrix of an input graph using the graph's `topology`, `node properties`, and `edge properties`. The goal of the GAE training objective will be to create an adjancecy matrix with each edge being the probability of existence of a certain edge. 

 We perform negative edge sampling on the input graphs: This essentially means that we'll also add random, 'noisy' edges to the input graph durining training and ensure that the model *does not* reconstruct those


While evaluating the model on the test set, we'll measure for overfitting by monitoring the area under the ROC curve, and the average precision. 

In [ ]:
# train_test.py

import torch
import torch.nn.functional as F
from torch_geometric.utils import negative_sampling
from sklearn.metrics import roc_auc_score, average_precision_score
from tqdm import tqdm
from torch.cuda.amp import autocast


def train(model, optimizer, data_loader, device):
    model.train()
    total_loss = 0

    for data in tqdm(data_loader, desc='Training'):
        data = data.to(device)
        optimizer.zero_grad()
        # Encode
        z = model.encode(data.x, data.edge_index, data.edge_attr[:, :-1])
        # Positive edges
        pos_edge_index = data.edge_index
        pos_edge_attr = data.edge_attr[:, :-1]

        # Negative edges (sampled for negative examples)
        neg_edge_index = negative_sampling(
            edge_index=pos_edge_index,
            num_nodes=data.num_nodes,
            num_neg_samples=pos_edge_index.size(1),
            method='sparse'
        )

        # For negative edges, we need to create dummy edge attributes (e.g., zeros)
        neg_edge_attr = torch.zeros(neg_edge_index.size(1), data.edge_attr.size(1)-1).to(device)

        # Decode positive edges
        pos_pred = model.decode(z, pos_edge_index, pos_edge_attr, data.batch)

        # Decode negative edges
        neg_pred = model.decode(z, neg_edge_index, neg_edge_attr, data.batch)

        # Create labels
        pos_label = torch.ones(pos_pred.size()).to(device)
        neg_label = torch.zeros(neg_pred.size()).to(device)

        # Concatenate predictions and labels
        preds = torch.cat([pos_pred, neg_pred], dim=0)
        labels = torch.cat([pos_label, neg_label], dim=0)

        # Compute loss
        loss = F.binary_cross_entropy(preds, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        torch.cuda.empty_cache()

    return total_loss / len(data_loader)

def test(model, data_loader, device):
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
        for data in tqdm(data_loader, desc='Testing'):
            data = data.to(device)

            # Encode
            z = model.encode(data.x, data.edge_index, data.edge_attr[:, :-1])

            # Positive edges
            pos_edge_index = data.edge_index
            pos_edge_attr = data.edge_attr[:, :-1]

            # Negative edges (sampled for negative examples)
            neg_edge_index = negative_sampling(
                edge_index=pos_edge_index,
                num_nodes=data.num_nodes,
                num_neg_samples=pos_edge_index.size(1),
                method='sparse'
            )

            # For negative edges, we need to create dummy edge attributes (e.g., zeros)
            neg_edge_attr = torch.zeros(neg_edge_index.size(1), data.edge_attr.size(1)-1).to(device)

            # Decode positive edges
            pos_pred = model.decode(z, pos_edge_index, pos_edge_attr, data.batch)
            pos_label = torch.ones(pos_pred.size()).to(device)

            # Decode negative edges
            neg_pred = model.decode(z, neg_edge_index, neg_edge_attr, data.batch)
            neg_label = torch.zeros(neg_pred.size()).to(device)

            # Collect predictions and labels
            preds.append(torch.cat([pos_pred, neg_pred], dim=0).cpu())
            labels.append(torch.cat([pos_label, neg_label], dim=0).cpu())

    preds = torch.cat(preds, dim=0).numpy()
    labels = torch.cat(labels, dim=0).numpy()

    # Compute ROC-AUC and Average Precision
    roc_auc = roc_auc_score(labels, preds)
    avg_precision = average_precision_score(labels, preds)

    return roc_auc, avg_precision


Let's instatiate the model and associated objects. We can also take a look at what the Guided GAE architecture looks like. The key daddition to this model is that it also allows the VGAE to use graph-wide edge properties to guide the reconstruction of the adjacency matrix.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

in_channels = benign_graphs[0].x.size(1)
hidden_channels = 256
edge_attr_dim = 3
global_emb_dim = 128

encoder = GATEncoderWithEdgeAttr(in_channels, hidden_channels, edge_attr_dim)
global_edge_embedding = GlobalEdgeEmbedding(edge_attr_dim, global_emb_dim)
decoder = DecoderWithGlobalEdge(hidden_channels, edge_attr_dim, global_emb_dim)

model = GAEWithGlobalEdge(encoder, decoder, global_edge_embedding).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.003)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 5)

model

We notice that the GAE has encoder and decoder blocks. 

The model architecture is composed of an encoder, a global edge embedding module, and a decoder, all integrated to form a Graph Autoencoder (GAE)-like framework that incorporates global edge-level context.

The encoder, defined by `GATEncoderWithEdgeAttr`, takes node and edge attributes and transforms them into latent node embeddings. It first applies a `GATv2Conv(32, 256, heads=4)`, which uses multi-headed attention (4 heads) to produce a total of `1024` features from `32` input features per node (4 heads × 256 features each). Unlike a standard GAT layer, this variant can incorporate edge attributes into the attention mechanism, allowing the attention weights to depend on both node and edge-level information. The resulting embeddings are normalized with a `BatchNorm1d(1024)` operation to stabilize training. Next, a `GraphUNet(1024, 256, 256, depth=3, pool_ratios=[0.5, 0.5, 0.5])` is applied, providing a hierarchy of pooling and unpooling steps to capture multi-scale graph structure and produce refined 256-dimensional embeddings. This multi-resolution representation helps the encoder capture both local connectivity patterns and more global topological features.

A `GlobalEdgeEmbedding` module provides a global context of edge information. This module takes a 3-dimensional global edge feature vector and processes it through a small MLP: a `Linear(3, 128)` layer, followed by a `ReLU` activation, and then another `Linear(128, 128)`. The output is a 128-dimensional global edge embedding that encapsulates global edge attributes, complementing the node-level embeddings by providing a holistic view of the entire graph’s edge structure.

The decoder, implemented as `DecoderWithGlobalEdge`, integrates the node embeddings, edge embeddings, and global edge embedding to reconstruct or predict edges. The raw edge attributes (3-dimensional) are first mapped to 256 dimensions via `fc_edge` (`Linear(3,256)`). The global edge embedding (128-dimensional) is similarly expanded to 256 dimensions by `fc_global` (`Linear(128,256)`). The node embeddings, edge embeddings, and global edge embeddings are then combined to form a 768-dimensional vector (256 per component). The decoder processes this combined vector through `fc1` (`Linear(768,768)`), which can further refine the joint representation, and then `fc` (`Linear(768,1)`), which outputs a scalar value for each potential edge. This scalar can represent probabilities or scores for edge existence, thereby enabling the model to reconstruct the adjacency structure or predict edge properties.


Now that we've instantiated the requisite pieces, we can run training for a few epochs. 

In [ ]:
# Example training and testing loop

from tqdm import tqdm 

for epoch in range(1, 10):
    print(f'#############Epoch {epoch}###############')
    train_loss = train(model, optimizer, train_loader, device)
    auc, ap = test(model, test_loader, device)
    scheduler.step()
    print(f'AUC: {auc} AP: {ap} Loss: {train_loss}')

In [ ]:
torch.save(model.state_dict(), "artifacts/sample_weights.pth")

## Step 4: Evaluate Model Performance on Attack Data

In this final step, we aim to evaluate the performance of the trained model on previously unseen benign and attack data, which had been loaded into our `attack_loader` in prior steps. We'll evaluate `False Positive Rates (FPR)`, `True Positive Rates (TPR)`, and the `Confusion Matrix` for binary classification. In our example, anomalous edges have a label `1` and benign edges `0`. 

Recall that a VGAE outputs probilities of an edge occuring in the adjacency matrix. From there, anomaly detection is straightforward. We can threshold the liklihood of a given edge as belonging to a graph or not. Any edge with a low probability can be classified as anomalous.

First, we'll define a function to compute the metrics given a data loader and model. 

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from cuml.metrics import roc_auc_score, confusion_matrix
from sklearn.metrics import precision_recall_curve, roc_curve, auc, balanced_accuracy_score
import numpy as np
import cupy as cp
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve

# Define the function to compute metrics
def compute_metrics(data_loader, model, device, threshold=0.1):
    all_labels = []
    all_preds = []

    model.eval()
    with torch.no_grad():
        for data in tqdm(data_loader, desc='Inference examples'):
            data = data.to(device)
            z = model.encode(data.x, data.edge_index, data.edge_attr[:, :-1])
            preds = model.decode(z, data.edge_index, data.edge_attr[:, :-1], data.batch)
            # Likelihood of Not Belonging in Graph
            all_preds.append(1 - cp.asarray(preds.float().cpu().numpy()))
            all_labels.append(cp.asarray(data.edge_attr[:, -1].cpu().numpy()))
    
    all_preds = cp.concatenate(all_preds)
    all_labels = cp.concatenate(all_labels)
    
    unique_values, counts = cp.unique(all_labels, return_counts=True)

    print(f"Unique values: {unique_values}")
    print(f"Counts: {counts}")
    
    fpr, tpr, thresholds = roc_curve(all_labels.get(), all_preds.get())
    
    roc_auc = roc_auc_score(all_labels.get(), all_preds.get())

    del all_preds
    del all_labels

    plt.figure()
    plt.plot(fpr, tpr, color='blue', label='ROC curve (area = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], color='red', linestyle='--')  # Diagonal line
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.show()

    youdens_j = tpr - fpr
    idx = np.argmax(youdens_j)
    best_threshold = thresholds[idx]
    best_fpr = fpr[idx]
    best_tpr = tpr[idx]

    return roc_auc, best_fpr, best_tpr

Finally, we can compute metrics for the VGAE on mixed attack and benign data.

In [ ]:
compute_metrics(attack_loader, model, device, threshold=0.0015)